In [1]:
from pathlib import Path
import re
import pandas as pd

In [2]:
df = pd.read_csv("stratqa_with_gpt_responses.csv")

In [5]:
for col in df.columns:
    df[col] = df[col].str.lower()

In [6]:
df.head()

,question,answer,babbage_no_cot,babbage_cot,davinci_no_cot,davinci_cot
0,are more people today related to genghis khan ...,yes,a: so the answer is yes.,a: no. genghis khan was the founder of the mon...,a: so the answer is yes.,a: genghis khan lived from 1162 to 1227. juliu...
1,could the members of the police perform lawful...,no,a: so the answer is no.,a: the police are a police force. police force...,a: so the answer is yes.,a: the members of the police are not members o...
2,would a monoamine oxidase candy bar cheer up a...,no,a: so the answer is no.,a: a monoamine oxidase candy bar is a candy ba...,a: so the answer is no.,a: a monoamine oxidase inhibitor is a type of ...
3,would a dog respond to bell before grey seal?,yes,a: so the answer is yes.,a: the dog would not respond to the bell befor...,a: so the answer is no.,a: a dog has a faster reaction time than a gre...
4,is a pound sterling valuable?,no,a: so the answer is yes.,a: the pound sterling is worth about $1.60. th...,a: so the answer is yes.,a: a pound sterling is a unit of currency. cur...


In [7]:
def compute_accuracy(df, no_cot_col, cot_col, answer_col):
    no_cot_answers = df[no_cot_col].to_list()
    cot_answers = df[cot_col].to_list()
    answers = df[answer_col].str.lower()
    answers = answers.to_list()
    answers = zip(no_cot_answers, cot_answers, answers)
    answers = list(answers)
    answers = [[str(a[0]), str(a[1]), str(a[2])] for a in answers]
    answers = [
        [
            re.findall(answer[2], answer[0]),
            re.findall(answer[2], answer[1]),
            answer[2],
        ]
        for answer in answers
    ]
    answers = [[len(answer[0]), len(answer[1]), answer[2]] for answer in answers]
    no_cot = 0
    cot = 0
    for a, b, c in answers:
        if a != 0:
            no_cot += 1
        if b != 0:
            cot += 1

    no_cot_percent = no_cot / len(answers) * 100
    cot_percent = cot / len(answers) * 100

    print(f"No COT accuracy: {no_cot_percent}%")
    print(f"COT accuracy: {cot_percent}%")

In [8]:
compute_accuracy(df, "babbage_no_cot", "babbage_cot", "answer")

No COT accuracy: 48.8%
COT accuracy: 56.49999999999999%


In [10]:
compute_accuracy(df, "davinci_no_cot", "davinci_cot", "answer")

No COT accuracy: 59.4%
COT accuracy: 70.1%
